In [1]:
!pip install transformers
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
from os import truncate
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from transformers import TFBertForSequenceClassification
from matplotlib import pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 10.6 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 86 kB 3.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Tensorflow version 2.8.2
Running on TPU  ['10.122.9.186:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.122.9.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.122.9.186:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

from sklearn.model_selection import KFold

# Set the split ratio to 9:1 for training and test set
def split_test_set(df):
  x, train_set = train_test_split(df, 
        stratify=df['label'],
        test_size=0.1, 
        random_state=42)
  return x, train_set

# Read file
df_raw = pd.read_excel("classifier1.xlsx",names=["query","intent","label"])
# df_raw = pd.read_excel("classifier2.xlsx",names=["query","intent","label"])
# df_raw = pd.read_excel("classifier3.xlsx",names=["query","intent","label"])
# label

# Add another column y in the dataframe for the numeric labels 
# Classifier 1
df_label = pd.DataFrame({"label":["G","D"],"y":list(range(2))})
# Classifier 2
# df_label = pd.DataFrame({"label":["PG","G"],"y":list(range(2))})
# Classifier 3
# df_label = pd.DataFrame({"label":["PD","ND","D"],"y":list(range(3))})
df_raw = pd.merge(df_raw,df_label,on="label",how="left")

# Randomly select a subset of data from each user intent
size = 20       # sample size
replace = True  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
df_raw = df_raw.groupby('intent', as_index=False).apply(fn)

# Split the training set and the test set
x, test_set = split_test_set(df_raw)
# Apply 5-fold cross validation for the training set
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# Get training sets and validation sets
split_list = list(kf.split(x))
train_1 = df_raw.iloc[split_list[0][0]]
validation_1 = df_raw.iloc[split_list[0][1]]
train_2 = df_raw.iloc[split_list[1][0]]
validation_2 = df_raw.iloc[split_list[1][1]]
train_3 = df_raw.iloc[split_list[2][0]]
validation_3 = df_raw.iloc[split_list[2][1]]
train_4 = df_raw.iloc[split_list[3][0]]
validation_4 = df_raw.iloc[split_list[3][1]]
train_5 = df_raw.iloc[split_list[4][0]]
validation_5 = df_raw.iloc[split_list[4][1]]


# Encode the texts, samples include user intents
def encode_with(ds, limit=-1):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.take(limit)
    for index, row in ds.iterrows():
        query = row["query"]
        label = row["y"]
        intent = row["intent"]
        bert_input = tokenizer.encode_plus(query,intent,add_special_tokens=True,max_length=20,
                                 padding='max_length',return_attention_mask=True,truncation='only_first')
         
        
        bert_input = tokenizer.encode_plus(query,add_special_tokens=True,max_length=20,padding='max_length',return_attention_mask=True,truncation=True) 
  
        
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    input_ids=np.array(input_ids_list)
    attention_masks=np.array(attention_mask_list)
    labels=np.array(label_list)
    token_types=np.array(token_type_ids_list)
      
    return input_ids,attention_masks,labels,token_types

# Encode the texts, for classifiers that only use queries
def encode_without(ds, limit=-1):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.take(limit)
    for index, row in ds.iterrows():
        query = row["query"]
        label = row["y"]
        
        bert_input = tokenizer.encode_plus(query,add_special_tokens=True,max_length=20,padding='max_length',return_attention_mask=True,truncation=True) 
  
        
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    input_ids=np.array(input_ids_list)
    attention_masks=np.array(attention_mask_list)
    labels=np.array(label_list)
    token_types=np.array(token_type_ids_list)
      
    return input_ids,attention_masks,labels,token_types

# Encode training set and validation set
def encode_process(trainset, validationset, without):
  print("Encoding processing")
  if without == False:
    train_inp,train_mask,train_label,train_type = encode_with(trainset)
    val_inp,val_mask,val_label,val_type=encode_with(validationset)
  else:
    train_inp,train_mask,train_label,train_type = encode_without(trainset)
    val_inp,val_mask,val_label,val_type=encode_without(validationset)
  return train_inp, train_mask, train_label, train_type, val_inp, val_mask, val_label, val_type

# encode training and validation sets in each fold, with user intents
train_inp1,train_mask1,train_label1,train_type1,val_inp1,val_mask1,val_label1,val_type1=encode_process(train_1,validation_1,False)
train_inp2,train_mask2,train_label2,train_type2,val_inp2,val_mask2,val_label2,val_type2=encode_process(train_2,validation_2,False)
train_inp3,train_mask3,train_label3,train_type3,val_inp3,val_mask3,val_label3,val_type3=encode_process(train_3,validation_3,False)
train_inp4,train_mask4,train_label4,train_type4,val_inp4,val_mask4,val_label4,val_type4=encode_process(train_4,validation_4,False)
train_inp5,train_mask5,train_label5,train_type5,val_inp5,val_mask5,val_label5,val_type5=encode_process(train_5,validation_5,False)

# encode training and validation sets in each fold, without user intents
train_inp1_wo,train_mask1_wo,train_label1_wo,train_type1_wo,val_inp1_wo,val_mask1_wo,val_label1_wo,val_type1_wo=encode_process(train_1,validation_1,True)
train_inp2_wo,train_mask2_wo,train_label2_wo,train_type2_wo,val_inp2_wo,val_mask2_wo,val_label2_wo,val_type2_wo=encode_process(train_2,validation_2,True)
train_inp3_wo,train_mask3_wo,train_label3_wo,train_type3_wo,val_inp3_wo,val_mask3_wo,val_label3_wo,val_type3_wo=encode_process(train_3,validation_3,True)
train_inp4_wo,train_mask4_wo,train_label4_wo,train_type4_wo,val_inp4_wo,val_mask4_wo,val_label4_wo,val_type4_wo=encode_process(train_4,validation_4,True)
train_inp5_wo,train_mask5_wo,train_label5_wo,train_type5_wo,val_inp5_wo,val_mask5_wo,val_label5_wo,val_type5_wo=encode_process(train_5,validation_5,True)

test_inp, test_mask, test_label, test_type = encode_with(test_set)
test_inp_wo, test_mask_wo, test_label_wo, test_type_wo = encode_without(test_set)

print("train_inp1 length: {}, train_mask length: {}".format(train_inp1.shape,train_mask1.shape))
print(tokenizer.decode(train_inp1[1]))
print("train_label1 length: {}".format(train_label1.shape))
print("val_inp1 length: {}, val_label1 length: {}".format(val_inp1.shape,val_label1.shape))
print("test_inp length: {}, test_label length: {}".format(test_inp.shape,test_label.shape))



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Encoding processing
Encoding processing
Encoding processing
Encoding processing
Encoding processing
Encoding processing
Encoding processing
Encoding processing
Encoding processing
Encoding processing
train_inp1 length: (446, 20), train_mask length: (446, 20)
[CLS] how old is a child movie ticket [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
train_label1 length: (446, 1)
val_inp1 length: (112, 20), val_label1 length: (112, 1)
test_inp length: (62, 20), test_label length: (62, 1)


In [6]:
# Set the hyperparameters
learning_rate = 3e-5
number_of_epochs = 1
batch_size = 32
with tpu_strategy.scope():

# Model initialization with user intents
  model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
  # model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  # model.layers[1].trainable = False

  model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

  # Model initialization without user intents
  model_wo = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
  # model_wo = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

  optimizer_wo = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  loss_wo = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric_wo = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  # model_wo.layers[1].trainable = False
  model_wo.compile(optimizer=optimizer_wo, loss=loss_wo, metrics=[metric_wo])


print("model with summary")
print(model.summary())

print("model without summary")
print(model_wo.summary())

# Fit model
print("Training process, with usesr intents")
bert_history = model.fit([train_inp1,train_mask1,train_type1],train_label1,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp1,val_mask1,val_type1],val_label1))
bert_history = model.fit([train_inp2,train_mask2,train_type2],train_label2,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp2,val_mask2,val_type2],val_label2))
bert_history = model.fit([train_inp3,train_mask3,train_type3],train_label3,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp3,val_mask3,val_type3],val_label3))
bert_history = model.fit([train_inp4,train_mask4,train_type4],train_label4,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp4,val_mask4,val_type4],val_label4))
bert_history = model.fit([train_inp5,train_mask5,train_type5],train_label5,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp5,val_mask5,val_type5],val_label5))

print("Training process, without user intents")
bert_history = model_wo.fit([train_inp1_wo,train_mask1_wo,train_type1_wo],train_label1_wo,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp1_wo,val_mask1_wo,val_type1_wo],val_label1_wo))
bert_history = model_wo.fit([train_inp2_wo,train_mask2_wo,train_type2_wo],train_label2_wo,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp2_wo,val_mask2_wo,val_type2_wo],val_label2_wo))
bert_history = model_wo.fit([train_inp3_wo,train_mask3_wo,train_type3_wo],train_label3_wo,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp3_wo,val_mask3_wo,val_type3_wo],val_label3_wo))
bert_history = model_wo.fit([train_inp4_wo,train_mask4_wo,train_type4_wo],train_label4_wo,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp4_wo,val_mask4_wo,val_type4_wo],val_label4_wo))
bert_history = model_wo.fit([train_inp5_wo,train_mask5_wo,train_type5_wo],train_label5_wo,batch_size=batch_size,epochs=number_of_epochs,validation_data=([val_inp5_wo,val_mask5_wo,val_type5_wo],val_label5_wo))


print('Test set evaluation, with user intents')
model.evaluate([test_inp,test_mask,test_type],test_label)
print('Test set evaluation, without user intents')
model_wo.evaluate([test_inp_wo,test_mask_wo,test_type_wo],test_label_wo)



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model with summary
Model: "tf_bert_for_sequence_classification_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_265 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None
model without summary
Model: "tf_bert_for_sequence_classification_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                 

[0.03989721089601517, 0.9838709235191345]